# Vector representation of DeCS codes

Primero hay que descargar los dataset con el script `get_data.py` que está en la raíz del repositorio.

In [1]:
import sys
sys.path.append("..")
import mesinesp2.tokenizer

Cargamos un diccionario con todos los códigos DeCS

In [2]:
descriptions = mesinesp2.tokenizer.get_descriptions("../data/raw/DeCS2020.obo", tokenize_definition = False, tokenize_name = False)
decs = {}
for key,val in descriptions.items():
    if key.startswith("D"):
        code = key
        document = val["name"]
        if "def" in val:
            document = document + " " + val["def"]
        decs[code] = document

In [3]:
decs

{'DeCS': 'DeCS',
 'D': 'COMPUESTOS QUÍMICOS Y DROGAS',
 'D000001': 'Calcimicina Un antibiótico poliéter, ionóforo, de Streptomyces chartreusensis. Se une y transporta cationes a través de las membranas y desacopla la fosforilación oxidativa mientras inhibe la ATPasa de la mitocondria hepática del ratón. Esta sustancia es utilizada principalmente como herramienta bioquímica para estudiar el papel de los cationes divalentes en varios sistemas biológicos.',
 'D000002': 'Temefós Un insecticida organotiofosforado.',
 'D000003': 'Mataderos Lugares donde los animales son sacrificados y preparados para el mercado.',
 'D000004': 'Abreviaturas como Asunto 1. f. Abreviación gráfica formada por el conjunto de letras iniciales de una expresión compleja; p. ej., ONU por Organización de las Naciones Unidas, ovni por objeto volador no identificado, IPC por índice de precios al consumo. 2. f. Cada una de las letras de una sigla (‖ abreviación formada por letras iniciales). P. ej., O, N y U son siglas e

Si necesitas normalizar y tokenizar los textos existen las funciones `mesinesp2.tokenizer.normalizer()` y `mesinesp2.tokenizer.tokenizer()`. Por ejemplo así tokenizamos los decs y los normalizamos.

In [4]:
mesinesp2.tokenizer.tokenizer(decs["D000003"], split_sentences=False, is_df = False, normalize = True)

['mataderos',
 'lugares',
 'donde',
 'los',
 'animales',
 'son',
 'sacrificados',
 'y',
 'preparados',
 'para',
 'el',
 'mercado']

La idea es generar un diccionario que tenga como key el código DeCS y como value el vector que representa ese código, algo como esto.

```
{
   'D000002' : array([0.01, 0.03, ... 0.07, 0.09]) #   El largo del arreglo debiese ser igual a las dimensiones de 
                                                       los embeddings que elijas para representar los códigos
    ...
}
```

In [5]:
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer

In [6]:
import nltk
from nltk.corpus import stopwords
stop_words = stopwords.words('spanish')

In [7]:
for i in decs.keys():
    decs[i] = mesinesp2.tokenizer.tokenizer(decs[i], split_sentences=False, is_df = False, normalize = True)

In [8]:
for i in decs.keys():
    decs[i] = [word for word in decs[i] if word not in stop_words]

In [9]:
decs['D000003']

['mataderos', 'lugares', 'animales', 'sacrificados', 'preparados', 'mercado']

### TF-IDF

In [10]:
import gensim # module for computing word embeddings
import numpy as np # linear algebra module
import sklearn.feature_extraction # package to perform tf-idf vertorization
import json

C:\Users\carol\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [11]:
def vectorize_text(sentences, idf_location):
    tfidfvectorizer = sklearn.feature_extraction.text.TfidfVectorizer() # instance of the tf-idf vectorizer
    tfidfvectorizer.fit(sentences) # fitting the vectorizer and transforming the properties
    idf = {key:val for key,val in zip(tfidfvectorizer.get_feature_names(),tfidfvectorizer.idf_)}
    with open(idf_location, 'w+', encoding='utf-8') as json_file:
        json.dump(idf, json_file, indent=2, ensure_ascii=False)

In [12]:
sent = []
for sentence in decs.values():
    sent.append(' '.join(sentence))

In [13]:
vectorize_text(sent, r'C:/Users/carol/Desktop/Practica 2/tfidf.json')

In [14]:
with open(r'C:/Users/carol/Desktop/Practica 2/tfidf.json') as f:
    tfidf = json.load(f)

### Embeddings

In [15]:
from gensim.models.keyedvectors import KeyedVectors

In [16]:
wordvectors_vec = 'fasttext-sbwc.3.6.e20.vec'
cantidad = 100000
wordvectors = KeyedVectors.load_word2vec_format(r'C:\Users\carol\Desktop\Practica 2\SBW-vectors-300-min5.txt', limit = cantidad)

In [21]:
import fasttext.util

mix = fasttext.load_model(r'C:\Users\carol\Desktop\Practica 2\mix_fasttext.bin')

In [22]:
def to_vector(texto,model,idf):
    """ Receives a sentence string along with a word embedding model and 
    returns the vector representation of the sentence"""
    tokens = texto.split() # splits the text by space and returns a list of words
    vec = np.zeros(300) # creates an empty vector of 300 dimensions
    for word in tokens: # iterates over the sentence
        if (word in model) & (word in idf): # checks if the word is both in the word embedding and the tf-idf model
            vec += model[word]*idf[word] # adds every word embedding to the vector
    if np.linalg.norm(vec) > 0:
        return vec / np.linalg.norm(vec) # divides the vector by their normal
    else:
        return vec

In [23]:
vectorized_sent_word2vec = [to_vector(text, wordvectors, tfidf) for text in sent]

In [33]:
dictionary_SBW = {list(decs.keys())[i]: vectorized_sent_word2vec[i].tolist() for i in range(len(list(decs.keys())))}    

In [25]:
vectorized_sent_fasttext = [to_vector(text, mix, tfidf) for text in sent]

In [34]:
dictionary_mix = {list(decs.keys())[i]: vectorized_sent_fasttext[i].tolist() for i in range(len(list(decs.keys())))}

In [35]:
with open('decs_sbw.json', 'w') as fp:
    json.dump(dictionary_SBW, fp)

In [36]:
with open('decs_mix.json', 'w') as fp:
    json.dump(dictionary_mix, fp)